In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# **Data Pre Processing**

In [ ]:
# Importing the required libraries
import re
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/emotion_dataset.csv')
df.head()

,Unnamed: 0,Emotion,Text,Clean_Text
0,0,neutral,Why ?,NaN
1,1,joy,Sage Act upgrade on my to do list for tommorow.,Sage Act upgrade list tommorow
2,2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...,WAY HOMEGIRL BABY FUNERAL MAN HATE FUNERALS SH...
3,3,joy,Such an eye ! The true hazel eye-and so brill...,eye true hazel eyeand brilliant Regular feat...
4,4,joy,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...,ugh babe hugggzzz u babe naamazed nga ako e...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34792 entries, 0 to 34791
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  34792 non-null  int64 
 1   Emotion     34792 non-null  object
 2   Text        34792 non-null  object
 3   Clean_Text  34326 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [ ]:
df.shape

(34792, 4)

In [ ]:
Sentences = df['Clean_Text']
Sentiments = df['Emotion']

In [ ]:
len(Sentences), len(Sentiments)

(34792, 34792)

In [ ]:
print(df['Emotion'].value_counts(),'\n')

Emotion
joy         11045
sadness      6722
fear         5410
anger        4297
surprise     4062
neutral      2254
disgust       856
shame         146
Name: count, dtype: int64 



In [ ]:
print(f"Number of null sentiment: {df['Emotion'].isnull().sum()}\n")

Number of null sentiment: 0



In [ ]:
print(f"Number of null content: {df['Clean_Text'].isnull().sum()}\n")

Number of null content: 466



In [ ]:
# Drop rows with null values in the 'Clean_Text' column
df = df.dropna(subset=['Clean_Text'])

In [ ]:
print(f"Number of null content: {df['Clean_Text'].isnull().sum()}\n")

Number of null content: 0



In [ ]:
def preprocess(sentences):
    # Ensure sentences is a pandas Series
    if isinstance(sentences, pd.Series):
        # Truncate sentences to 300 characters
        sentences = sentences.str[:300]

        # Replace HTML line breaks with space
        sentences = sentences.str.replace(r"<br\s*/?>", " ", regex=True)

        # Replace non-alphabetic characters (except apostrophes) with space
        sentences = sentences.str.replace(r"[^a-zA-Z']", " ", regex=True)

        # Convert to lowercase
        sentences = sentences.str.lower()

    else:
        print("not pd series")
    return sentences


df['Clean_Text'] = preprocess(df['Clean_Text'])

In [ ]:
print(df['Emotion'].value_counts(),'\n')

Emotion
joy         11037
sadness      6719
fear         5409
anger        4286
surprise     4062
neutral      1811
disgust       856
shame         146
Name: count, dtype: int64 



In [ ]:
# Remove rows where emotion is 'disgust' or 'shame'
df = df[~df['Emotion'].isin(['disgust', 'shame'])]

In [ ]:
# Randomly sample 4100 rows where emotion is 'joy'
joy_rows = df[df['Emotion'] == 'joy']
sampled_joy_rows = joy_rows.sample(n=4100, random_state=42)  # Use a random_state for reproducibility

# Drop the sampled rows from the original DataFrame
df = df.drop(sampled_joy_rows.index)

In [ ]:
print(df['Emotion'].value_counts(),'\n')

Emotion
joy         6937
sadness     6719
fear        5409
anger       4286
surprise    4062
neutral     1811
Name: count, dtype: int64 



In [ ]:
df.head(10)

,Unnamed: 0,Emotion,Text,Clean_Text
2,2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...,way homegirl baby funeral man hate funerals sh...
3,3,joy,Such an eye ! The true hazel eye-and so brill...,eye true hazel eyeand brilliant regular feat...
4,4,joy,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...,ugh babe hugggzzz u babe naamazed nga ako e...
5,5,fear,I'm expecting an extremely important phonecall...,im expecting extremely important phonecall min...
6,6,sadness,.Couldnt wait to see them live. If missing th...,couldnt wait live missing nh wasnt painful en...
7,7,surprise,maken Tip 2: Stop op een moment dat je het hel...,maken tip stop op een moment dat je het hel...
8,8,surprise,En dan krijg je ff een cadeautje van een tweep...,en dan krijg je ff een cadeautje van een tweep...
9,9,surprise,@1116am Drummer Boy bij op verzoek van @BiemO...,drummer boy bij op verzoek van welke ...
10,10,anger,The bull tossed the effigy out of their hands ...,bull tossed effigy hands infuriated
11,11,sadness,People hide their behind a #fake smile.,people hide fake smile


In [ ]:
# Install all necessary libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and text handling
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter

# Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Deep Learning libraries (PyTorch)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

# Transformers for BERT
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset

# Word Embeddings


# NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization Settings
sns.set(style='whitegrid')

# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords and punkt (tokenizer)
nltk.download('stopwords')
nltk.download('punkt')


# Define the stopwords removal function
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())
    filtered_text = ' '.join([word for word in word_tokens if word not in stop_words])
    return filtered_text

# Apply the function to the 'description' column
df['Clean_Text'] = df['Clean_Text'].apply(remove_stopwords)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df.shape

(29224, 4)

In [ ]:
# Assuming 'df' is your DataFrame and you want to drop 'column1' and 'column2'
df = df.drop(['Unnamed: 0', 'Text'], axis=1)


In [ ]:
print(f"Number of null content: {df['Clean_Text'].isnull().sum()}\n")

Number of null content: 0



In [ ]:
# Export the DataFrame to a CSV file
df.to_csv('sahilFInalDataset.csv', index=False)


In [ ]:
df=pd.read_csv('/content/sahilFInalDataset.csv')

In [ ]:
print(f"Number of null content: {df['Clean_Text'].isnull().sum()}\n")

Number of null content: 54



In [ ]:
# Drop rows with null values in the 'Clean_Text' column
df = df.dropna(subset=['Clean_Text'])

In [ ]:
print(f"Number of null content: {df['Clean_Text'].isnull().sum()}\n")

Number of null content: 0



In [ ]:
df.head(10)

,Emotion,Clean_Text
0,sadness,way homegirl baby funeral man hate funerals sh...
1,joy,eye true hazel eyeand brilliant regular featur...
2,joy,ugh babe hugggzzz u babe naamazed nga ako e ba...
3,fear,im expecting extremely important phonecall min...
4,sadness,couldnt wait live missing nh wasnt painful enu...
5,surprise,maken tip stop op een moment dat je het hele p...
6,surprise,en dan krijg je ff een cadeautje van een tweep...
7,surprise,drummer boy bij op verzoek van welke uitvoerin...
8,anger,bull tossed effigy hands infuriated
9,sadness,people hide fake smile


## **Bert Training**

In [ ]:
# Load the pre-trained BERT tokenizer and model
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')
model= BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6) # 6 sentiment classes



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Encode the text data using the tokenizer
encoded_data= tokenizer.batch_encode_plus(df['Clean_Text'].tolist(), max_length=128, padding=True, truncation=True, return_tensors='pt')

In [ ]:
import torch
from sklearn.model_selection import train_test_split
# Split the data into training and test sets for BERT
input_ids= encoded_data['input_ids']
attention_masks= encoded_data['attention_mask']
# Encode the Score column
labels = torch.tensor(df['Emotion'].map({'joy': 0, 'sadness': 1, 'fear': 2, 'anger': 3, 'surprise': 4, 'neutral': 5}).values)

# Split the data into training and test sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)


In [ ]:
# Convert your data into the required format for the Dataset object
def convert_to_dataset(input_ids, attention_masks, labels):
    dataset = Dataset.from_dict({
        'input_ids': input_ids.numpy(),
        'attention_mask': attention_masks.numpy(),
        'labels': labels.numpy()
    })
    return dataset

In [ ]:
# Create the training and validation datasets
train_dataset= convert_to_dataset(train_inputs, train_masks, train_labels)
val_dataset= convert_to_dataset(val_inputs, val_masks, val_labels)

In [ ]:
# Use a data collator to handle padding
data_collator= DataCollatorWithPadding(tokenizer)

# Set up the training arguments
training_args= TrainingArguments(output_dir='./bert_output_sahil', num_train_epochs=20, per_device_train_batch_size=256, per_device_eval_batch_size=128, warmup_steps=500, weight_decay=0.01, logging_dir='./logs', report_to= 'none')

In [ ]:
# Create the trainer
trainer= Trainer(model= model, args= training_args, train_dataset= train_dataset, eval_dataset= val_dataset, data_collator= data_collator, tokenizer= tokenizer)

# Train the model
trainer.train()

In [ ]:
# Evaluate the BERT model
model.eval()
predictions= []
true_labels= []

In [ ]:
# Create a DataLoader for the validation set
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
# Make predictions on the validation set
with torch.no_grad():
    for batch in val_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
        outputs = model(**inputs)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = batch[2].cpu().numpy()
        predictions.extend(np.argmax(logits, axis=1))
        true_labels.extend(label_ids)

In [ ]:
# Calculate evaluation metrics for BERT
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'BERT - Accuracy: {accuracy:.2f}')
print(f'BERT - Precision: {precision:.2f}')
print(f'BERT - Recall: {recall:.2f}')
print(f'BERT - F1 Score: {f1:.2f}')

BERT - Accuracy: 0.92
BERT - Precision: 0.92
BERT - Recall: 0.92
BERT - F1 Score: 0.92


In [ ]:
!pip install transformers huggingface_hub


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
model.save_pretrained("dissertation_sahil_bert")
tokenizer.save_pretrained("dissertation_sahil_bert")

('dissertation_sahil_bert/tokenizer_config.json',
 'dissertation_sahil_bert/special_tokens_map.json',
 'dissertation_sahil_bert/vocab.txt',
 'dissertation_sahil_bert/added_tokens.json')

In [ ]:
model.push_to_hub('dissertation_sahil_bert')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahadev23/dissertation_sahil_bert/commit/247231f7824382f30361862dd5c04840d082c5fd', commit_message='Upload BertForSequenceClassification', commit_description='', oid='247231f7824382f30361862dd5c04840d082c5fd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("dissertation_sahil_bert")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahadev23/dissertation_sahil_bert/commit/f6ca8aa22e20d68a537e48a0260a26ecc1d7ccd4', commit_message='Upload tokenizer', commit_description='', oid='f6ca8aa22e20d68a537e48a0260a26ecc1d7ccd4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords and punkt (tokenizer)
nltk.download('stopwords')
nltk.download('punkt')


# Define your model and tokenizer
model_name = "mahadev23/dissertation_sahil_bert"  # Replace with your model repository name
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model = model.to(device)
model.eval()

def predict_single_string(text, tokenizer, model, device):
    # Tokenize the input text
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,  # Adjust MAX_LEN based on your model's configuration
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()

    # Convert logits to predicted label
    predicted_label = np.argmax(logits, axis=1)

    return predicted_label[0]
#'fear': 0, 'anger': 1, 'joy': 2,'sadness':3}
# Example usage:
text = "He died in a accident."

def preprocess_sentence(sentence):
    # Truncate sentence to 300 characters
    sentence = sentence[:300]

    # Replace HTML line breaks with space
    sentence = re.sub(r"<br\s*/?>", " ", sentence)

    # Replace non-alphabetic characters (except apostrophes) with space
    sentence = re.sub(r"[^a-zA-Z']", " ", sentence)

    # Convert to lowercase
    sentence = sentence.lower()

    # Tokenize (split by spaces)
    tokens = sentence.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    cleaned_sentence = ' '.join(tokens)

    return cleaned_sentence


# Preprocess the sentence
cleaned_text = preprocess_sentence(text)

print(cleaned_text)

predicted_class = predict_single_string(cleaned_text, tokenizer, model, device)
print(f"Predicted class: {predicted_class}")
#{'joy': 0, 'sadness': 1, 'fear': 2, 'anger': 3, 'surprise': 4, 'neutral': 5})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

died accident
Predicted class: 1
